# Send Requests

## Launch a server

This code uses `subprocess.Popen` to start an SGLang server process, equivalent to executing 

```bash
python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
--port 30000 --host 0.0.0.0 --log-level warning
```
in your command line and wait for the server to be ready.

In [9]:
import subprocess
import time
import requests
import signal
import os

server_process = subprocess.Popen(
    [
        "python",
        "-m",
        "sglang.launch_server",
        "--model-path",
        "meta-llama/Meta-Llama-3.1-8B-Instruct",
        "--port",
        "30000",
        "--host",
        "0.0.0.0",
        "--log-level",
        "error",
    ],
    text=True,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

while True:
    try:
        response = requests.get(
            "http://localhost:30000/v1/models",
            headers={"Authorization": "Bearer None"},
        )
        if response.status_code == 200:
            break
    except requests.exceptions.RequestException:
        time.sleep(1)

print("Server is ready. Proceeding with the next steps.")

Server is ready. Proceeding with the next steps.


## Send a Request

Once the server is running, you can send test requests using curl.

In [10]:
!curl http://localhost:30000/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer None" \
  -d '{"model": "meta-llama/Meta-Llama-3.1-8B-Instruct", "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "What is a LLM?"}]}'

{"id":"b0e50122fcd84d4bae744baa2f776f23","object":"chat.completion","created":1729809561,"model":"meta-llama/Meta-Llama-3.1-8B-Instruct","choices":[{"index":0,"message":{"role":"assistant","content":"LLM stands for Large Language Model. It's a type of artificial intelligence (AI) designed to process and generate human-like language. An LLM uses natural language processing (NLP) to analyze and understand the context, syntax, and semantics of language, allowing it to generate coherent and context-specific text.\n\nLarge Language Models are typically trained on massive datasets of text, which enables them to learn patterns, relationships, and structures of language. This training allows them to:\n\n1. Understand and interpret language inputs\n2. Generate text based on given prompts or topics\n3. Answer questions and provide information on a wide range of subjects\n4. Engage in conversation, using context and understanding to respond to queries\n\nSome common applications of LLMs include:\

## Using OpenAI Compatible API

SGLang supports OpenAI-compatible APIs. Here are Python examples:

In [11]:
import openai

# Always assign an api_key, even if not specified during server initialization.
# Setting an API key during server initialization is strongly recommended.

client = openai.Client(
    base_url="http://127.0.0.1:30000/v1", api_key="None"
)

# Chat completion example

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant"},
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)
print(response)

ChatCompletion(id='219e5ecd446a41bd82572094ccdfe88f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here are 3 countries and their capitals:\n\n1. **Country:** Japan\n**Capital:** Tokyo\n\n2. **Country:** Australia\n**Capital:** Canberra\n\n3. **Country:** Brazil\n**Capital:** Brasília', refusal=None, role='assistant', function_call=None, tool_calls=None), matched_stop=128009)], created=1729809563, model='meta-llama/Meta-Llama-3.1-8B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=46, prompt_tokens=49, total_tokens=95, prompt_tokens_details=None))


In [12]:
def terminate_process(process):
    try:
        process.terminate()
        try:
            process.wait(timeout=5)
        except subprocess.TimeoutExpired:
            if os.name != 'nt':
                pgid = os.getpgid(process.pid)
                os.killpg(pgid, signal.SIGKILL)
            else:
                process.kill()
            process.wait()
            
    except Exception as e:
        print(f"Error terminating process: {e}")
        try:
            process.kill()
            process.wait()
        except:
            pass

terminate_process(server_process)
time.sleep(2)

## Use Embedding Models



In [13]:
# Start a new server process for embedding models
# Equivalent to running this in the shell:
# python -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-7B-instruct --port 30010 --host 0.0.0.0 --is-embedding --log-level error
embedding_process = subprocess.Popen(
    [
        "python",
        "-m",
        "sglang.launch_server",
        "--model-path",
        "Alibaba-NLP/gte-Qwen2-7B-instruct",
        "--port",
        "30010",
        "--host",
        "0.0.0.0",
        "--is-embedding",
        "--log-level",
        "error",
    ],
    text=True,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

while True:
    try:
        response = requests.get(
            "http://localhost:30010/v1/models",
            headers={"Authorization": "Bearer None"},
        )
        if response.status_code == 200:
            break
    except requests.exceptions.RequestException:
        time.sleep(1)

print("Embedding server is ready. Proceeding with the next steps.")

Embedding server is ready. Proceeding with the next steps.


## Use Curl

In [14]:
# Get the first 10 elements of the embedding

! curl -s http://localhost:30010/v1/embeddings \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer None" \
  -d '{"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "input": "Once upon a time"}' \
  | python3 -c "import sys, json; print(json.load(sys.stdin)['data'][0]['embedding'][:10])"

[0.0083160400390625, 0.0006804466247558594, -0.00809478759765625, -0.0006995201110839844, 0.0143890380859375, -0.0090179443359375, 0.01238250732421875, 0.00209808349609375, 0.0062103271484375, -0.003047943115234375]


## Using OpenAI Compatible API

In [15]:
import openai

client = openai.Client(
    base_url="http://127.0.0.1:30010/v1", api_key="None"
)

# Text embedding example
response = client.embeddings.create(
    model="Alibaba-NLP/gte-Qwen2-7B-instruct",
    input="How are you today",
)

embedding = response.data[0].embedding[:10]
print(embedding)

[0.00603485107421875, -0.0190582275390625, -0.01273345947265625, 0.01552581787109375, 0.0066680908203125, -0.0135955810546875, 0.01131439208984375, 0.0013713836669921875, -0.0089874267578125, 0.021759033203125]


In [16]:
terminate_process(embedding_process)
time.sleep(2)